In [ ]:
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup

import pymongo

from datetime import datetime

In [ ]:
# read data from GHMC website
url = "https://www.ghmc.gov.in/covid_details.aspx"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [ ]:
list_rows = [] # to append cleaned data
rows = soup.find_all('tr') # read all table rows

# data cleanup
for row in rows:
    row_details = (row.find_all('th') or row.find_all('td'))
    str_cells = str(row_details)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text().replace("\n","").replace("[","").replace("]","")
    list_rows.append(cleantext)
    
final_df = pd.DataFrame(list_rows) # append rows to data frame

final_df = final_df[0].str.split(',', expand=True) # split data to columns 
final_df.fillna('Unavailable', inplace = True)

# make first row as header 
new_header = final_df.iloc[0] 
final_df = final_df[1:] 
final_df.columns = new_header 
final_df.rename(columns={'Mandal Name': 'Mandal'}, inplace = True)

final_df.reset_index(drop = True, inplace = True)
final_df.columns = final_df.columns.str.strip()
final_df.drop(['Unavailable'], axis = 1, inplace = True)

final_df = final_df.apply(lambda x: x.astype(str).str.lstrip())
final_df = final_df.apply(lambda x: x.astype(str).str.capitalize())

final_df.Gender = final_df.Gender.replace('M', "Male")
final_df.Gender = final_df.Gender.replace('F', "Female")
final_df.Gender = final_df.Gender.replace('', "N/A")
final_df.Gender = final_df.Gender.replace('-', "N/A")

today = datetime.today()
final_df['data_extract_date'] = today

In [ ]:
# push to mongo cluster

client = pymongo.MongoClient("<DB Connection link><login details>")
db = client.covid19hyderabad
covid19hyderabad = db.covid19hyderabad
covid19hyderabad.insert_many(final_df.to_dict('records'))